<img style="float: left;padding-right: 10px" width ="40px" src="https://raw.githubusercontent.com/bartczernicki/DecisionIntelligence.GenAI.Workshop/main/Images/SemanticKernelLogo.png">

## Semantic Kernel - Decisions with Native Functions 

Decision Intelligence applied in this module:  
* Selecting of a Decision-mMking framework dynamically using native code  
* Decision Scenario: Using a Monte Carlo Simulation to provide an estimate of decsision uncertainty  
* Provide a range for Decision Uncertainty using Confidence Intervals 

Semantic Kernel Functions are the core building blocks of functionality for intelligent AI orchestration. Semantic Kernel functions usually have a single responsibility to perform. For example, a Sementic Kernel function can: send an e-mail, call an API, recommend a reasoning framework for a high-stakes decision etc. 

What makes Semantic Kernel functions so special? Can't one just produce similar outcomes by using a prompt in an LLM or just writing a software program? In order to answer that, one has to look at what makes GenAI so innovative. The GenAI models have a unique ability to process instructions with reasoning and logic. This allows these models to behave almost like a human decision maker. Even with basic prompts, GenAI models perform reasonably well. However, providing GenAI models with additional functions allows them to gain access to business processes, data and basically anything a GenAI model should consider when performing AI orchestration. 

For example, imagine you want to prepare a great Thanksgiving dinner and want to get help from a GenAI cooking application to create a new recipe. You enter what you want todo and it comes up with the most delicious looking feast for Thanksgiving. However, there is a problem it used ingredients and recommended using cooking appliances that you do not own! You could enter all of the ingredients and keep prompting until it narrowed down the what you could realistically make. Wouldn't it be better if the GenAI cooking application had access to: your available ingredients, your time availability, kitchen appliances and even your allergic preferences. Now the GenAI model can craft a Thanksgiving feast understanding the parameters and data without having to be guided. This is exactly what Semantic Kernel functions provide.

Semantic Kernel functions come in two flavors: semantic functions and native functions. Native Functions provide the ability for Semantic Kernel to orchestrate plans, agents and reasoning paths using native functions (methods) of the language runtimes of C#, Python or Java. This allows AI architects to re-purpose existing business processes, APIs and other complex tasks that require a programming language. The image below illustrates the core value of native functions.  

<img style="display: block; margin: auto;" width ="400px" src="https://learn.microsoft.com/en-us/semantic-kernel/media/native-function-explainer.png">

For example, you may want to expose a native function that has access to a database or calls an API into business data. This is not possible currently with semantic functions and AI prompts.  

Learn more about Semantic Kernel Native Functions: https://learn.microsoft.com/en-us/semantic-kernel/agents/plugins/using-the-kernelfunction-decorator?tabs=Csharp 

### Step 1 - Initialize Configuration Builder & Build the Semantic Kernel Orchestration 

Execute the next two cells to:
* Use the Configuration Builder to load the API secrets.  
* Use the API configuration to build the Semantic Kernel orchestrator.

In [1]:
#r "nuget: Microsoft.Extensions.Configuration.Json, 9.0.4"
#r "nuget: Microsoft.SemanticKernel, 1.47"
#r "nuget: Microsoft.SemanticKernel.Plugins.Core, 1.47-alpha"

using Microsoft.Extensions.Configuration;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using Microsoft.SemanticKernel.Connectors.OpenAI;
using System.IO;

var configurationBuilder = new ConfigurationBuilder()
    .SetBasePath(Directory.GetCurrentDirectory())
    .AddJsonFile("local.settings.json", optional: true, reloadOnChange: true)
    .AddJsonFile("secrets.settings.json", optional: true, reloadOnChange: true);
var config = configurationBuilder.Build();

// IMPORTANT: You ONLY NEED either Azure OpenAI or OpenAI connectiopn info, not both.
// Azure OpenAI Connection Info
var azureOpenAIEndpoint = config["AzureOpenAI:Endpoint"];
var azureOpenAIAPIKey = config["AzureOpenAI:APIKey"];
var azureOpenAIModelDeploymentName = config["AzureOpenAI:ModelDeploymentName"];
// OpenAI Connection Info 
var openAIAPIKey = config["OpenAI:APIKey"];
var openAIModelId = config["OpenAI:ModelId"];

Installed Packages Microsoft.Extensions.Configuration.Json, 9.0.4 Microsoft.SemanticKernel, 1.47.0 Microsoft.SemanticKernel.Plugins.Core, 1.47.0-alpha

In [2]:
Kernel semanticKernel;

// Set the flag to use Azure OpenAI or OpenAI. False to use OpenAI, True to use Azure OpenAI
var useAzureOpenAI = true;

// Create a new Semantic Kernel instance
if (useAzureOpenAI)
{
    Console.WriteLine("Using Azure OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddAzureOpenAIChatCompletion(
            deploymentName: azureOpenAIModelDeploymentName,
            endpoint: azureOpenAIEndpoint,
            apiKey: azureOpenAIAPIKey)
        .Build();
}
else
{
    Console.WriteLine("Using OpenAI Service");
    semanticKernel = Kernel.CreateBuilder()
        .AddOpenAIChatCompletion(
            modelId: openAIModelId,
            apiKey: openAIAPIKey)
        .Build();
}

Using Azure OpenAI Service


### Step 2 - Create a Simple Native Function in Semantic Kernel

Execute the cell below to create a very simple native function that uses a C# inline method. Notice the native function takes no parameters. It retrieves the name of a productivity decision framework to use. In this case that return name is hard-coded to "Price's Law".

Execute the cell below to invoke the function. Note the function return instantly, as it is not calling any GenAI service. This function is simply being invoked by the Semantic Kernel wrapper.  

In [3]:
// Create a function from an inline (lambda) method
var nameOfProductivityFramework = semanticKernel.CreateFunctionFromMethod(() => "Price's Law", "GetNameOfProductivityFramework", "Retrieves the name of the Productivity Framework to use.");

// Invoke the function using Semantic Kernel
var response = await semanticKernel.InvokeAsync(nameOfProductivityFramework);
Console.WriteLine(response);

Price's Law


### Step 3 - Create a Native Function in Semantic Kernel with Dynamic Parameters

Execute the cell below to create a native function that takes a parameter as input. This shows that C# native functions can have different execution paths. The execution paths can obviously be quite complex. Basically, any C# logic flow will work. 

In [4]:
// Create a function from an inline (lambda) method with parameters
var nameOfProductivityFramework = semanticKernel.CreateFunctionFromMethod((string typeOfProductivity) => (typeOfProductivity == "Sales") ? "Price's Law" : "Pareto Principle", "GetNameOfProductivityFramework", "Retrieves the name of the Productivity Framework to use.");

// Pass the "Sales" parameter to the function
var kernelArguments = new KernelArguments()
{
    ["typeOfProductivity"] = "Sales"
};
var response = await semanticKernel.InvokeAsync(nameOfProductivityFramework, kernelArguments);
Console.WriteLine(response);

// Pass the "Other" parameter to the function
var kernelArgumentsOther = new KernelArguments()
{
    ["typeOfProductivity"] = "Other"
};
var responseOther = await semanticKernel.InvokeAsync(nameOfProductivityFramework, kernelArgumentsOther);
Console.WriteLine(responseOther);

Price's Law
Pareto Principle


### Step 4 - Use Native Functions To Simulate the Uncertainty of a Decision  

> 📜 "The best way to predict the future is to simulate it. And the best way to simulate it is with Monte Carlo."
>
> -- <cite>Nassim Nicholas Taleb (Lebanese-American essayist, scholar, best known for his work on probability)</cite>

For more complex decisions, native functions can use statitics, advanced probabilistic algorithms, analytics, machine learning, AI etc. that have been relied on in software for decades. One such method is Monte Carlo Simulations. These powerful Monte Carlo simulation techniques are used everywhere: risk management, sports gambling, medical decision-making, game theory, energy market forecasting etc.  In simple terms, a Monte Carlo simulation is basically a series of many runs testing different plausable parameters. Running a Monte Carlo simulation many times results in an output of a plausible range.  

An simple use-case for a Monte Carlo simulation is to provide a realistic range for an average probability. Imagine you want to illustrate the uncertainty of a decision that you have calculated to be 70% successful. On "average" the probability of success can be interpreted as 70%. However, what is the range of possible succcessesi if that 70% decision model is run 100x? A Monte Carlo simulation can help solve that answer.

Run the cell below to create a new KernelFunction that will take in the confidence parameter and output a simple string with the lower and upper bounds of a 95% confidence interval. A 95% Confidence Interval output will tell us if we execute this calculate decision 100x (times) that has a 70% success probability, what is the realistic range of success in those 100x (times).

In [5]:
// Add the System.ComponentModel namespace to use the Description attribute
using System.ComponentModel;

[KernelFunction]
public string RetrieveConfidenceIntervalMonteCarlo(
    [Description("Claimed Probability Percentage")] int probability)
{
    const int NUMBEROFSIMULATIONS = 100000; // 100,000 simulations, make this smaller for faster results
    Console.WriteLine($"Simulating {NUMBEROFSIMULATIONS:n0} iterations with a claimed decision confidence of {probability}%...");

    var random = new Random();  // Add seed for reproducibility
    var bootstrapConfidenceScores = new List<double>();
    for (int i = 0; i != NUMBEROFSIMULATIONS; i++) // Bootstrap Simulations (bootstrap estimates)
    {
        var bootstrapSample = new List<double>();
        for (int j = 0; j != 100; j++)
        {
            var randomIndex = random.Next(0, 100);

            if (randomIndex < probability)
            {
                bootstrapSample.Add(1);
            }
        }

        bootstrapConfidenceScores.Add(bootstrapSample.Count());
    }

    // Sort the confidence scores to calculate the percentiles
    var bootstrapConfidenceScoresSorted = bootstrapConfidenceScores.OrderBy(a => a).ToList();
    // Calculate the 2.5% and 97.5% percentiles
    var lowerPercentileIndex = Convert.ToInt32(0.025 * NUMBEROFSIMULATIONS);
    var topPercentileIndex = Convert.ToInt32(0.975 * NUMBEROFSIMULATIONS);

    var lowerPercentile = Math.Round(bootstrapConfidenceScoresSorted[lowerPercentileIndex], 3);
    var upperPercentile = Math.Round(bootstrapConfidenceScoresSorted[topPercentileIndex], 3);

    var confidenceUncertaintyRange = $"95% Confidence Interval of a {probability}% success model: {lowerPercentile} to {upperPercentile} successful decision outcomes of 100 decisions made.";
    return confidenceUncertaintyRange;
}

Run the cell below to invoke the native Kernel function. This will run a Monte Carlo Simulation with 100,000 simulations of a decision with a confidence (probability) of 70% being run 100x. Basically, this will provide the uncertainty range if you had made 100 decisions of the same 70% success decision. 

In [6]:
var retrieveConfidenceInterval = semanticKernel.CreateFunctionFromMethod(
    RetrieveConfidenceIntervalMonteCarlo, "RetrieveConfidenceIntervalMonteCarlo", 
    "Generates Confidence Interval from provided Confidence Percentage");

// Change the probability to another integer and invoke the function, to see other Confidence Intervals 
var kernelArgumentsConfidence70 = new KernelArguments()
{
    ["probability"] = 70
};
var confidenceIntervalRange70 = semanticKernel.InvokeAsync(retrieveConfidenceInterval, kernelArgumentsConfidence70); 
Console.WriteLine(confidenceIntervalRange70.Result);

Simulating 100,000 iterations with a claimed decision confidence of 70%...
95% Confidence Interval of a 70% success model: 61 to 79 successful decision outcomes of 100 decisions made.
